# Task 2g Solution

In [1]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import pickle

In [2]:
detection_sci = np.load('data/detection_sci.npy') # read detection sci image
detection_wht = np.load('data/detection_wht.npy') # read detection wht image

--- cut out a portion of the image for analysis

In [3]:
x = 800 # x-center of the image
y = 800 # y-center of the image
r = 200 # width/2 of cutout, must be int

In [4]:
detection_sci = detection_sci[x-r:x+r, y-r:y+r] # cutout a portion of the science image
detection_wht = detection_wht[x-r:x+r, y-r:y+r] # cutout a portion of the weight image

--- define the noise in each pixel and make a significance map (signal/noise)

In [5]:
detection_noise = 1./np.sqrt(detection_wht) #Â conversion from weight to noise
detection_sig = detection_sci/detection_noise # signifance map

--- now run segmentation on the image

In [6]:
from photutils import detect_sources

In [7]:
threshold = 5
npixels = 10

In [8]:
segm = detect_sources(detection_sig, threshold, npixels)

In [9]:
from photutils import SourceCatalog, CircularAperture, aperture_photometry

In [10]:
cat = SourceCatalog(detection_sci, segm)

--- get a list of positions (x,y) of the sources

In [11]:
positions = []
for obj in cat:
    positions.append(np.transpose((obj.xcentroid, obj.ycentroid)))

--- make a CicrcularAperture object. This can be plotted but is mostly used for the aperture photometry.

In [12]:
aper_r = 5. # radius of aperture in pixels
apertures = CircularAperture(positions, aper_r)

--- now open all the individual filters

In [13]:
image_dir = 'data'

In [14]:
filters = [ 'f090w', 'f150w', 'f200w','f277w','f356w','f444w']

In [15]:
aper_cat = {}

In [16]:
for f in filters:

    # --- read in the science image and slice for our region
    sci = fits.getdata(f'{image_dir}/{f}_sci.fits')[x-r:x+r, y-r:y+r]
    # --- run aperture photometry to get photometry table
    phot_table = aperture_photometry(sci, apertures)
    # --- convert aperture flux column to an array and place in output dictionary
    aper_cat[f+'_signal'] = np.array(phot_table['aperture_sum'])

    # --- read in the weight image and slice for our region
    wht = fits.getdata(f'{image_dir}/{f}_wht.fits')[x-r:x+r, y-r:y+r]
    # --- define the noise image
    noise = 1/np.sqrt(wht)
    # --- square this ready for summing
    noisesq = noise**2
    # --- run aperture photometry to get photometry table
    phot_table = aperture_photometry(noisesq, apertures)
    # --- convert aperture flux column to an array and place in output dictionary
    aper_cat[f+'_noise'] = np.sqrt(np.array(phot_table['aperture_sum']))
    print(phot_table)

 id      xcenter            ycenter            aperture_sum    
           pix                pix                              
--- ------------------ ------------------ ---------------------
  1  338.3291036911487  11.50775604398455 0.0011310737341209277
  2   365.150727622869 1.8688983580609198 0.0009024904920271341
  3 355.90514274903336  7.489960537779359  0.001226168799115673
  4 324.66582080216966  8.647914669720532 0.0011826018297984538
  5  117.0994843661324  9.447650666476859 0.0011723752486259423
  6 122.04342955192479 18.796027567161953 0.0012381679331792555
  7  179.2170351383231 18.371438047300128 0.0011924560115754684
  8  327.2584581903846  23.79832133537443 0.0012053713418724334
  9 299.77440612725576 22.400314887856485 0.0012141506473909178
 10  220.3601381625638  36.49981068349711 0.0012140105126635758
...                ...                ...                   ...
116 177.41296607489843 356.83149267524675  0.001161985233471448
117 187.90895131683982  377.796584719505

In [17]:
pickle.dump(aper_cat, open('data/aper_cat.p','wb'))